In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import plotly.graph_objects as go
import ipywidgets as widgets

from axisproj import optimal, objective, make_basis

In [2]:
%load_ext autoreload

# For retina displays
%config InlineBackend.figure_format ='retina'

In [3]:
def prepare(data, method, knn):
    if method == 'lpp':
        X = data.T
        obj = objective.LPPObjective(knn=12,sigma=0.3)
        labs = None
    elif method == 'lde':
        X = data[:, :-1].T
        labs = np.ravel(data[:,-1])
        obj = objective.LDEObjective(knn=12, labs=labs)
    elif method == 'pca':
        X = data.T
        obj = objective.PCAObjective()
        labs = None
    else:
        print(f'unknown method: {method}')
        return
    
    X = StandardScaler().fit_transform(X.T).T
    
    return X, obj, labs

###  seawater

In [4]:
data = pd.read_csv(f'../data/seawater.csv')
knn = 12
method = 'lpp'
X, obj, labs = prepare(data.values, method, knn)
colors = X[51,:]
colorscale = 'Viridis'
showscale=True

### wine

In [8]:
data = pd.read_csv(f'../data/wine.csv')
knn = 12
method = 'lde'
X, obj, labs = prepare(data.values, method, knn)
colors = (labs-1)/2
colorscale = [[0, 'red'], [0.5, 'blue'], [1, 'green']]
showscale=False

### climate

In [60]:
data = pd.read_csv(f'../data/climate.csv')
knn = 12
method = 'lde'
X, obj, labs = prepare(data.values, method, knn)
colors = labs
colorscale = [[0, 'red'], [1, 'blue']]
showscale=False

### Compute Optimal Projections

In [9]:
%%time
lp, ap = optimal(X, obj, normalize=False)
print(f'lp:{len(lp)} ap:{len(ap)}')

lp:2 ap:2
CPU times: user 433 ms, sys: 79.5 ms, total: 513 ms
Wall time: 133 ms


### Linear Projections

In [10]:
for i, p in enumerate(lp):
    V = p['V']
    Y = X.T.dot(V)
    
    t1 = go.Scatter(x=Y[:,0], y=Y[:,1], mode="markers", marker=dict(size=5, color=colors,  colorscale=colorscale, showscale=showscale))
    f1 = go.FigureWidget(go.Figure(data=[t1]))   
    
    t2 = go.Bar(y=p['histogram'])
    f2 = go.FigureWidget(go.Figure(data=[t2]))   
    print(f'LP:{i}')
    display(widgets.HBox([f1, f2]))   

LP:0


    'data': [{'marker': {'color': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,…

LP:1


    'data': [{'marker': {'color': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,…

### Axis Aligned Projections

In [11]:
dim = list(data)
for a in ap:
    alpha =  a['alpha']
    V = make_basis(X.shape[0], alpha)
    Y = X.T.dot(V)
    t1 = go.Figure(data=[go.Scatter(x=Y[:,0], y=Y[:,1], mode="markers", marker=dict(size=5, color=colors,  colorscale=colorscale, showscale=showscale))],
                  layout=go.Layout(
                      xaxis_title=dim[alpha[0]], 
                      yaxis_title=dim[alpha[1]]
                  ))
    f1 = go.FigureWidget(t1)
    
    t2 = go.Figure(data=[go.Bar(y=a['histogram'])])
    f2 = go.FigureWidget(t2)
    print(f'LP:{a["LP"]} alpha:{alpha}')
    display(widgets.HBox([f1, f2]))   

LP:[0] alpha:(9, 12)


    'data': [{'marker': {'color': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,…

LP:[1] alpha:(0, 12)


    'data': [{'marker': {'color': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,…